In [3]:
import pandas as pd
import streamlit as st
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep




#Pergunta ao usuário qual site gostaria de realizar as análises dos relatórios
ativos = input('Escolha ativos da bolsa de valores que deseja identificar seus indicadores: ')

#Retira os espaços em branco dos ativos e deixa-os todos com letra maiúscula
ativos_formatados = [ativo.strip().upper() for ativo in ativos.split(",")] 
print(f'Seus ativos são: {ativos_formatados}')

# Configuração do WebDriver (Chrome neste caso)
#Criando um serviço responsável por instalar a versão do chrome driver de forma automática
service = Service(ChromeDriverManager().install())
#Executando este serviço
navegador = webdriver.Chrome(service=service)

#Entrando no site statusinvest
navegador.get('https://statusinvest.com.br/')
sleep(15)

#Clicando para fechar a propaganda
navegador.find_element(By.XPATH, '/html/body/div[19]/div/div/div[1]/button/i').click()


#Para ativo dentro da lista de ativos que o usuário digitou:
for ativo in ativos_formatados:
    
    #clicando na bússula de pesquisa
    navegador.find_element(By.XPATH, '//*[@id="main-nav-nav"]/div/div[2]/div/ul/li[2]/a/i').click()
    #Colocando o ativo dentro da barra de pesquisa
    navegador.find_element(By.XPATH, '//*[@id="main-search"]/div[1]/span[1]/input[2]').send_keys(ativo)
    sleep(3)
    #Clicando no ativo desejado
    navegador.find_element(By.XPATH, '//*[@id="main-search"]/div[2]/div/div/a').click()
    #Identificando os principais indicadores da página
    dividend_yield = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[4]/div/div[1]/strong').text
    p_pv = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[2]/div/div[1]/strong').text
    numero_cotistas = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[6]/div/div[1]/strong').text
    valor_mercado = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[2]/div/div[2]/span[2]').text
    valor_caixa = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[3]/div/div[2]/span[2]').text
    valor_patrimonio_total = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[1]/div/div[2]/span[2]').text
    print(f'Seu ativo é {ativo} e seu principais indicadores são os seguintes: ')
    print(f'{ativo} possui {dividend_yield}% de dividend yield dos últimos 12 meses')
    print(f'{ativo} possui um P/PV de {p_pv}')
    print(f'{ativo} possui {numero_cotistas} cotistas atualmente')
    print(f'{ativo} possui {valor_mercado} de valor de mercado')
    print(f'{ativo} possui um total de {valor_caixa} em seu caixa')
    print(f'{ativo} possui um valor patrimonial total de {valor_patrimonio_total}')



# XPATH Para identificar se é um FII -> //*[@id="main-header"]/div[2]/div/div[1]/div/ol/li[2]/a/span
# XPATH Para identificar se é uma Ação -> //*[@id="main-header"]/div[2]/div/div[1]/div/ol/li[2]/a/span


# XPATH de dividend yield dos últimos 12 meses -> //*[@id="main-2"]/div[2]/div[1]/div[4]/div/div[1]/strong
# XPATH de P/PV -> //*[@id="main-2"]/div[2]/div[5]/div/div[2]/div/div[1]/strong
# XPATH de números de cotistas-> //*[@id="main-2"]/div[2]/div[5]/div/div[6]/div/div[1]/strong
# XPATH de Valor de mercado -> //*[@id="main-2"]/div[2]/div[5]/div/div[2]/div/div[2]/span[2]
# XPATH de Valor em caixa -> //*[@id="main-2"]/div[2]/div[5]/div/div[3]/div/div[2]/span[2]
# XPATH de Patrimônio total -> //*[@id="main-2"]/div[2]/div[5]/div/div[1]/div/div[2]/span[2]







            


Seus ativos são: ['HCTR11', 'HGLG11', 'IRDM11']
Seu ativo é HCTR11 e seu principais indicadores são os seguintes: 
HCTR11 possui 14,60% de dividend yield dos últimos 12 meses
HCTR11 possui um P/PV de 0,27
HCTR11 possui 160.564 cotistas atualmente
HCTR11 possui R$ 654.796.589 de valor de mercado
HCTR11 possui um total de R$ 14.250.211,61 em seu caixa
HCTR11 possui um valor patrimonial total de R$ 2.459.765.585
Seu ativo é HGLG11 e seu principais indicadores são os seguintes: 
HGLG11 possui 8,36% de dividend yield dos últimos 12 meses
HGLG11 possui um P/PV de 1,00
HGLG11 possui 486.864 cotistas atualmente
HGLG11 possui R$ 5.336.747.471 de valor de mercado
HGLG11 possui um total de R$ 53.606.387,00 em seu caixa
HGLG11 possui um valor patrimonial total de R$ 5.335.073.437
Seu ativo é IRDM11 e seu principais indicadores são os seguintes: 
IRDM11 possui 13,88% de dividend yield dos últimos 12 meses
IRDM11 possui um P/PV de 0,77
IRDM11 possui 287.137 cotistas atualmente
IRDM11 possui R$ 2.373